In [1]:
import pandas as pd
import numpy as np
import requests
import numpy as np
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = './Tesseract/Tesseract.exe'
import os

In [2]:
Bool_OCR = False

In [3]:
def Flashnamesa(name):
    if name == 'LC': return 'Linguagens'
    elif name == 'CH': return 'Humanas'
    elif name == 'CN': return 'Natureza'
    else: return 'Matemática'

In [4]:
def tri_3pl_enem(theta, a, b, c):
    return c + (1 - c) / (1 + np.exp(-a * (theta - b)))

In [5]:
def find_quantile(c):
    return round(((1+c)/2)*100, 2)

In [6]:
def imageApi(code):
    code = str(str(code) + '.png')
    imagem = 'https://niedsonemanoel.com.br/enem/An%C3%A1lise%20de%20Itens/OrdenarPorTri/1.%20Itens%20BNI_/' + code
    return imagem

In [7]:
def get_prova_string(ano, co_prova, sg, cor):
    if ano == 2017:
      if co_prova in [201711,201710,201712,201709]:
          return str(str(Flashnamesa(sg))+' '+str(cor)+' REGULAR')
      else:
          return str(str(Flashnamesa(sg))+' '+str(cor)+' PPL')
    else: return str(str(Flashnamesa(sg))+' '+str(cor)+' REGULAR')

In [8]:
# Encontrando o valor de theta que resulta em targ
def find_theta(a, b, c, targ):
    left = -100
    right = 100
    tol = 1e-5
    if (targ == 0.6):
      target = find_quantile(c)/100
    else:
      target = targ
    while (right - left) / 2 > tol:
        theta = (left + right) / 2
        value = tri_3pl_enem(theta, a, b, c)
        if value > target:
            right = theta
        else:
            left = theta
    return theta * 100 + 500

In [9]:
def ocrImage(code):
    output_file='../1. Itens BNI_/TXT/'+str(code)+'.txt'
    if os.path.exists(output_file):
        with open(output_file, 'r', encoding='utf-8') as file:
            return file.read()

    code = 'https://raw.githubusercontent.com/NiedsonEmanoel/NiedsonEmanoel/main/enem/An%C3%A1lise%20de%20Itens/OrdenarPorTri/1.%20Itens%20BNI_/'+str(str(code) + '.png')
    try:
        response = requests.get(code)
        img_array = np.array(bytearray(response.content), dtype=np.uint8)

        # Decodificar a imagem usando o OpenCV
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        ocrT = str(pytesseract.image_to_string(img, lang='por'))

        # Salvar o resultado no arquivo .txt se o arquivo não existir
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(ocrT)
            print(f"Arquivo '{output_file}' criado com sucesso!")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        ocrT = 'N/A'
    return ocrT

In [10]:
def toENEM(dfItens, ano, provas):
    countt = 1

    dfItens = dfItens[dfItens['TP_CERTIFICACAO'] == 2]
    dfItens = dfItens[dfItens.CO_PROVA.isin(provas)]

    numero_de_linhas = len(dfItens)

    dfItens['ANO'] = ano
    dfItens['IN_ENCCEJA'] = 1
    dfItens['IN_ENEM'] = 0
    dfItens['OCRSearch'] = 'N/A'
    dfItens['CO_PROVA'] = dfItens.apply(lambda row: get_prova_string(row['ANO'], row['CO_PROVA'], row['SG_AREA'], row['TX_COR'],), axis=1)

    
    for i in dfItens.index:
        dfItens.loc[i, "theta_065"] = find_theta(
            dfItens.loc[i, "NU_PARAM_A"],
            dfItens.loc[i, "NU_PARAM_B"],
            dfItens.loc[i, "NU_PARAM_C"],
            (find_quantile(dfItens.loc[i, "NU_PARAM_C"])/100),
        )
        dfItens.loc[i, "theta_080"] = find_theta(
            dfItens.loc[i, "NU_PARAM_A"],
            dfItens.loc[i, "NU_PARAM_B"],
            dfItens.loc[i, "NU_PARAM_C"],
            0.80,
        )
        dfItens.loc[i, "theta_099"] = find_theta(
            dfItens.loc[i, "NU_PARAM_A"],
            dfItens.loc[i, "NU_PARAM_B"],
            dfItens.loc[i, "NU_PARAM_C"],
            0.99,
        )
        dfItens.loc[i, 'PercentEspAcerto'] = find_quantile(
            dfItens.loc[i, "NU_PARAM_C"]
        )
        dfItens.loc[i, 'imagAPI'] = imageApi(dfItens.loc[i, 'CO_ITEM'])
        if Bool_OCR == True:
            dfItens.loc[i, 'OCRSearch'] = ocrImage(dfItens.loc[i, 'CO_ITEM'])
        
        print(str(countt)+'/'+str(numero_de_linhas)+' - '+str(dfItens.loc[i, 'ANO']))
        countt=countt+1

    del dfItens['TP_CERTIFICACAO']


    dfItens.to_csv('ITENS_PROVA_'+str(ano)+'_ENCCEJA.csv', index=False, sep=";", encoding="latin-1")
    dfItens.to_excel('Helpers/Saida_Itens_Processados/'+'ITENS_PROVA_'+str(ano)+'_ENCCEJA.xlsx', index=False)
    return dfItens

In [11]:
r17 = toENEM(
    pd.read_csv('Itens Externos/ITENS_ENCCEJA/MICRODADOS_ENCCEJA_ITENS_PROVA.csv', sep=";", encoding="latin-1"),
    2017,
    [201711, 201710, 201712, 201709, 201723, 201722, 201724, 201721]
    )

r18 = toENEM(
    pd.read_csv('Itens Externos/ITENS_ENCCEJA/ITENS_PROVA_2018.csv', sep=",", encoding="latin-1"),
    2018,
    [201811,201812,201810,201809]
    )

r19 = toENEM(
    pd.read_csv('Itens Externos/ITENS_ENCCEJA/MICRODADOS_ENCCEJA_ITENS_PROVA_2019.csv', sep=",", encoding="latin-1"),
    2019,
    [201911,201912,201910,201909]
    )

r20 = toENEM(
    pd.read_csv('Itens Externos/ITENS_ENCCEJA/MICRODADOS_ENCCEJA_ITENS_PROVA_2020.csv', sep=",", encoding="latin-1"),
    2020,
    [202011,202012,202010,202009]
    )

1/240 - 2017
2/240 - 2017
3/240 - 2017
4/240 - 2017
5/240 - 2017
6/240 - 2017
7/240 - 2017
8/240 - 2017
9/240 - 2017
10/240 - 2017
11/240 - 2017
12/240 - 2017
13/240 - 2017
14/240 - 2017
15/240 - 2017
16/240 - 2017
17/240 - 2017
18/240 - 2017
19/240 - 2017
20/240 - 2017
21/240 - 2017
22/240 - 2017
23/240 - 2017
24/240 - 2017
25/240 - 2017
26/240 - 2017
27/240 - 2017
28/240 - 2017
29/240 - 2017
30/240 - 2017
31/240 - 2017
32/240 - 2017
33/240 - 2017
34/240 - 2017
35/240 - 2017
36/240 - 2017
37/240 - 2017
38/240 - 2017
39/240 - 2017
40/240 - 2017
41/240 - 2017
42/240 - 2017
43/240 - 2017
44/240 - 2017
45/240 - 2017
46/240 - 2017
47/240 - 2017
48/240 - 2017
49/240 - 2017
50/240 - 2017
51/240 - 2017
52/240 - 2017
53/240 - 2017
54/240 - 2017
55/240 - 2017
56/240 - 2017
57/240 - 2017
58/240 - 2017
59/240 - 2017
60/240 - 2017
61/240 - 2017
62/240 - 2017
63/240 - 2017
64/240 - 2017
65/240 - 2017
66/240 - 2017
67/240 - 2017
68/240 - 2017
69/240 - 2017
70/240 - 2017
71/240 - 2017
72/240 - 2017
7